In [8]:
from keras import layers,models
import numpy as np
from keras.applications.vgg16 import VGG16

In [2]:
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input,decode_predictions

In [47]:
img_path='F:\\study\\ml\\DataSet\\dogs_vs_cats\\creative_commons_elephant.jpg'

In [4]:
img=image.load_img(img_path,target_size=(224,224))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
x.shape

(1, 224, 224, 3)

In [5]:
x

array([[[[181., 203., 224.],
         [181., 203., 224.],
         [181., 203., 224.],
         ...,
         [173., 190., 210.],
         [173., 190., 210.],
         [172., 189., 209.]],

        [[181., 203., 224.],
         [181., 203., 224.],
         [180., 202., 223.],
         ...,
         [173., 190., 210.],
         [173., 190., 210.],
         [172., 189., 209.]],

        [[178., 203., 223.],
         [180., 205., 225.],
         [178., 203., 223.],
         ...,
         [174., 191., 211.],
         [173., 190., 210.],
         [172., 189., 209.]],

        ...,

        [[ 75.,  82.,  41.],
         [ 95., 104.,  61.],
         [ 98., 107.,  62.],
         ...,
         [ 96.,  96.,  68.],
         [114., 117.,  86.],
         [ 83.,  87.,  54.]],

        [[120., 135.,  80.],
         [ 99., 113.,  60.],
         [107., 119.,  71.],
         ...,
         [ 95.,  94.,  66.],
         [101., 102.,  71.],
         [ 95.,  99.,  66.]],

        [[120., 135.,  80.],
       

In [6]:
x=preprocess_input(x)
x

array([[[[120.061     ,  86.221     ,  57.32      ],
         [120.061     ,  86.221     ,  57.32      ],
         [120.061     ,  86.221     ,  57.32      ],
         ...,
         [106.061     ,  73.221     ,  49.32      ],
         [106.061     ,  73.221     ,  49.32      ],
         [105.061     ,  72.221     ,  48.32      ]],

        [[120.061     ,  86.221     ,  57.32      ],
         [120.061     ,  86.221     ,  57.32      ],
         [119.061     ,  85.221     ,  56.32      ],
         ...,
         [106.061     ,  73.221     ,  49.32      ],
         [106.061     ,  73.221     ,  49.32      ],
         [105.061     ,  72.221     ,  48.32      ]],

        [[119.061     ,  86.221     ,  54.32      ],
         [121.061     ,  88.221     ,  56.32      ],
         [119.061     ,  86.221     ,  54.32      ],
         ...,
         [107.061     ,  74.221     ,  50.32      ],
         [106.061     ,  73.221     ,  49.32      ],
         [105.061     ,  72.221     ,  48.32      ]],

In [9]:
model=VGG16(weights='imagenet')

In [10]:
preds=model.predict(x)

In [12]:
print('predict:',decode_predictions(preds,top=5)[0])

predict: [('n02504458', 'African_elephant', 0.83298635), ('n01871265', 'tusker', 0.14927106), ('n02504013', 'Indian_elephant', 0.012977002), ('n02408429', 'water_buffalo', 0.0047369567), ('n02403003', 'ox', 9.239731e-06)]


In [16]:
np.argmax(preds[0])

386

Grad_CAM

In [20]:
from keras import backend as k

In [17]:
african_elephant_output=model.output[:,386]
last_conv_layer=model.get_layer('block5_conv3')

In [23]:
grads=k.gradients(african_elephant_output,last_conv_layer.output)[0]

In [24]:
grads.shape

TensorShape([Dimension(None), Dimension(14), Dimension(14), Dimension(512)])

In [25]:
pooled_grads=k.mean(grads,axis=(0,1,2))

In [26]:
iterate=k.function([model.input],[pooled_grads,last_conv_layer.output[0]])
pooled_grads_value,conv_layer_output_value=iterate([x])

In [35]:
for i in range(512):
    conv_layer_output_value[:,:,1]*=pooled_grads_value[i]
    
heatmap=np.mean(conv_layer_output_value,axis=-1)

In [36]:
conv_layer_output_value.shape

(14, 14, 512)

In [37]:
heatmap.shape

(14, 14)

In [38]:
import matplotlib.pyplot as plt

In [39]:
heatmap=np.maximum(heatmap,0)
heatmap/=np.max(heatmap)
# plt.matshow(heatmap)

In [40]:
import cv2
img=cv2.imread(img_path)
heatmap=cv2.resize(heatmap,(img.shape[1],img.shape[0]))
heatmap=np.uint8(255*heatmap)
heatmap=cv2.applyColorMap(heatmap,cv2.COLORMAP_JET)
superimposed_img=heatmap*0.4+img

In [46]:
cv2.imwrite('F:\\study\\ml\\DataSet\\dogs_vs_cats\\elephant_cam2.jpg',superimposed_img)

True

In [49]:
# img2_path='F:\\study\\ml\\DataSet\\dogs_vs_cats\\elephant_cam2.jpg'
# img2=image.load_img(img2_path)
# img2